## **Model Tuning Notebook**

In [1]:
from numba import cuda
import tensorflow as tf

def clear_gpu_memory():
    device = cuda.get_current_device()
    device.reset() # dump the memory contents to free up the memory (it accumulates over the session)
    
# CUDA (Nvidia GPU Computing)
if len(tf.config.list_physical_devices('GPU')) > 0:
    gpus = tf.config.list_physical_devices('GPU')
    print("Num GPUs Available: ", len(gpus))
    
    clear_gpu_memory()
    
    tf.config.experimental.set_memory_growth(gpus[0], True)

    tf.config.set_logical_device_configuration(
    gpus[0], 
    [tf.config.LogicalDeviceConfiguration(memory_limit=8192)])  # limit to 4GB

    tf.compat.v1.disable_eager_execution()


Num GPUs Available:  1


In [2]:
%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.append('../')

from src.models.models import *
from src.utils.modeling import *
from src.utils.preproc import *

from keras.models import Model
from keras.layers import Dense, Flatten, Dropout
from keras.optimizers import Adam

from sklearn.metrics import classification_report

In [3]:
# because the utils in the src are designed to be run from the root of the project,
# and by default jupyter runs from the notebook directory we need to change the working directory to the root

def find_project_root(filename=".git"): # .git is located in the root of the project
    current_dir = os.getcwd()
    while current_dir != os.path.dirname(current_dir): # stops only when at the root (moves up 1 level each iteration)
        if filename in os.listdir(current_dir):
            return current_dir
        current_dir = os.path.dirname(current_dir)

project_root = find_project_root()
os.chdir(project_root)  # change the working directory to the project root

print("Project root:", project_root, "CWD:", os.getcwd())

Project root: d:\deep_learning_project CWD: d:\deep_learning_project


### **Binary Classification Models Fine-Tuning**

#### **Traditional**

#### **Pre-Trained**

**VGG 16**

In [4]:
def binary_classification_vgg16_model(input_shape):
    
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    
    # train some layers and freeze others
    for layer in base_model.layers[:-4]:
        layer.trainable = False
    for layer in base_model.layers[-4:]:
        layer.trainable = True

    model = Sequential()
    
    model.add(base_model)
    
    model.add(Flatten())
    
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(1, activation='sigmoid'))
    
    # low learning rate for fine tuning
    model.compile(optimizer=Adam(learning_rate=0.0001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [5]:
if len(tf.config.list_physical_devices('GPU')) > 0:
    clear_gpu_memory()

train_gen, val_gen, X_test, y_test, class_weights = preproc_pipeline(desired_magnification='200X', 
                                                    image_resolution=(224, 224), 
                                                    classification_type='binary')

vgg16 = binary_classification_vgg16_model((224, 224, 3))

fitted_vgg16 = train_model(train_gen, val_gen, vgg16, class_weights=class_weights, epochs=10)

test_loss, test_acc = fitted_vgg16.evaluate(X_test, y_test)
print(f'Test loss: ', test_loss)
print(f'Test accuracy: ', test_acc)

Epoch 1/10
44/44 [==============================] - ETA: 0s - batch: 21.5000 - size: 32.0000 - loss: 0.6080 - accuracy: 0.6619

c:\Users\vic\miniconda3\envs\tf\lib\site-packages\keras\engine\training_v1.py:2332: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


44/44 [==============================] - 58s 569ms/step - batch: 21.5000 - size: 32.0000 - loss: 0.6080 - accuracy: 0.6619 - val_loss: 0.5586 - val_accuracy: 0.6755
Epoch 2/10
44/44 [==============================] - 17s 366ms/step - batch: 21.5000 - size: 32.0000 - loss: 0.4588 - accuracy: 0.7663 - val_loss: 0.3790 - val_accuracy: 0.8146
Epoch 3/10
44/44 [==============================] - 16s 356ms/step - batch: 21.5000 - size: 32.0000 - loss: 0.4083 - accuracy: 0.8132 - val_loss: 0.5486 - val_accuracy: 0.7053
Epoch 4/10
44/44 [==============================] - 17s 370ms/step - batch: 21.5000 - size: 32.0000 - loss: 0.2893 - accuracy: 0.8736 - val_loss: 0.2648 - val_accuracy: 0.8775
Epoch 5/10
44/44 [==============================] - 16s 350ms/step - batch: 21.5000 - size: 32.0000 - loss: 0.3044 - accuracy: 0.8636 - val_loss: 0.2750 - val_accuracy: 0.8642
Epoch 6/10
44/44 [==============================] - 16s 358ms/step - batch: 21.5000 - size: 32.0000 - loss: 0.2395 - accuracy: 0.89

<i> Data augmentation clearly improves the model's performance. </i>

In [4]:
def multiclass_classification_vgg16_model(input_shape):
    
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    
    # train some layers and freeze others
    for layer in base_model.layers[:-7]:
        layer.trainable = False
    for layer in base_model.layers[-7:]:
        layer.trainable = True

    model = Sequential()
    
    model.add(base_model)
    
    model.add(Flatten())
    
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(8, activation='softmax'))
    
    # low learning rate for fine tuning
    model.compile(optimizer=Adam(learning_rate=0.0001),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [5]:
train_gen, val_gen, X_test, y_test, class_weights = preproc_pipeline(desired_magnification='200X', 
                                                    image_resolution=(224, 224), 
                                                    classification_type='multiclass',
                                                    use_data_augmentation=True,
                                                    augmented_images_per_image=6)

vgg16 = multiclass_classification_vgg16_model((224, 224, 3))

fitted_vgg16 = train_model(train_gen, val_gen, vgg16, class_weights=class_weights, epochs=30, early_stopping_patience=5)

get_classification_report(fitted_vgg16, X_test, y_test)

Number of training images before data augmentation: 1408
Number of training images after data augmentation: 9856
Epoch 1/30
308/308 [==============================] - ETA: 0s - batch: 153.5000 - size: 32.0000 - loss: 1.7375 - accuracy: 0.2993

c:\Users\vic\miniconda3\envs\tf\lib\site-packages\keras\engine\training_v1.py:2332: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


308/308 [==============================] - 199s 492ms/step - batch: 153.5000 - size: 32.0000 - loss: 1.7375 - accuracy: 0.2993 - val_loss: 1.6420 - val_accuracy: 0.3742
Epoch 2/30
308/308 [==============================] - 144s 466ms/step - batch: 153.5000 - size: 32.0000 - loss: 1.0866 - accuracy: 0.5148 - val_loss: 1.2366 - val_accuracy: 0.5232
Epoch 3/30
308/308 [==============================] - 147s 472ms/step - batch: 153.5000 - size: 32.0000 - loss: 0.7633 - accuracy: 0.6423 - val_loss: 0.9014 - val_accuracy: 0.6490
Epoch 4/30
308/308 [==============================] - 152s 491ms/step - batch: 153.5000 - size: 32.0000 - loss: 0.6147 - accuracy: 0.7111 - val_loss: 0.6449 - val_accuracy: 0.7152
Epoch 5/30
308/308 [==============================] - 176s 569ms/step - batch: 153.5000 - size: 32.0000 - loss: 0.4997 - accuracy: 0.7468 - val_loss: 0.7463 - val_accuracy: 0.7417
Epoch 6/30
308/308 [==============================] - 189s 612ms/step - batch: 153.5000 - size: 32.0000 - loss:

c:\Users\vic\miniconda3\envs\tf\lib\site-packages\keras\engine\training_v1.py:2356: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


              precision    recall  f1-score   support

           0       0.94      0.94      0.94        16
           1       0.96      0.85      0.90       135
           2       0.87      0.85      0.86        39
           3       0.65      0.88      0.75        25
           4       0.81      0.83      0.82        30
           5       0.79      0.95      0.86        20
           6       0.69      0.69      0.69        16
           7       0.91      1.00      0.95        21

    accuracy                           0.86       302
   macro avg       0.83      0.87      0.85       302
weighted avg       0.88      0.86      0.87       302

